In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import keras
from tensorflow import keras

2024-08-23 05:30:15.807137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 05:30:15.817775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 05:30:15.817806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 05:30:15.826026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 05:30:16.378062: W tensorflow/compiler/tf

In [ ]:
# GPU 장치 목록 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow는 GPU를 사용할 수 있습니다.")
    for gpu in gpus:
        print(f"사용 가능한 GPU: {gpu}")

    # gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    # print(gpus)
    # if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)
else:
    print("TensorFlow는 GPU를 사용할 수 없습니다.")

In [ ]:
# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [ ]:
# 데이터 전처리
x_train = x_train.reshape(-1, 32, 32, 3).astype('float32') / 255.0
x_test = x_test.reshape(-1, 32, 32, 3).astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [ ]:
def seperate_convolution(X, f, nx, ny, s1, s2):
    X = keras.layers.Conv2D(filters=nx, kernel_size=(f, f), strides=(s1, s1), padding='same')(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.Conv2D(filters=ny, kernel_size=(1, 1), strides=(s2, s2), padding='same')(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    return X

In [ ]:
# inputs = tf.keras.Input(shape=(32, 32, 3)) #imagenet이 아니라 cifar-100이기 때문에 로 값을 더 줄인다고 생각 7x7에 도달하면 그 떄 avg pool 진행

# first_conv = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')(inputs)
# first_conv = keras.layers.BatchNormalization()(first_conv)
# first_conv = keras.layers.Activation('relu')(first_conv)
# sep_conv_16_1 = seperate_convolution(first_conv, 3, 32, 64, 1, 1)
# sep_conv_8_1 = seperate_convolution(sep_conv_16_1, 3, 64, 128, 2, 1)
# sep_conv_8_2 = seperate_convolution(sep_conv_8_1, 3, 128, 128, 1, 1)
# sep_conv_4_1 = seperate_convolution(sep_conv_8_2, 3, 128, 256, 2, 1)
# sep_conv_4_2 = seperate_convolution(sep_conv_4_1, 3, 256, 256, 1, 1)
# sep_conv_2 = seperate_convolution(sep_conv_4_2, 3, 256, 512, 2, 1)
# for i in range(5):
#     sep_conv_2 = seperate_convolution(sep_conv_2, 3, 512, 512, 1, 1)
# sep_conv_1_1 = seperate_convolution(sep_conv_2, 3, 512, 1024, 2, 1)
# sep_conv_1_2 = seperate_convolution(sep_conv_1_1, 3, 1024, 1024, 1, 1)
# avg_pool = keras.layers.AveragePooling2D((1, 1), strides=(1, 1), padding='same')(sep_conv_1_2)
# avg_pool = keras.layers.Flatten()(avg_pool)
# fc = keras.layers.Dense(100, 'softmax')(avg_pool)

inputs = tf.keras.Input(shape=(32, 32, 3)) #imagenet이 아니라 cifar-100이기 때문에 로 값을 더 줄인다고 생각 7x7에 도달하면 그 떄 avg pool 진행

first_conv = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')(inputs)
first_conv = keras.layers.BatchNormalization()(first_conv)
first_conv = keras.layers.Activation('relu')(first_conv)
sep_conv_16_1 = seperate_convolution(first_conv, 3, 32, 64, 1, 1)
sep_conv_8_1 = seperate_convolution(sep_conv_16_1, 3, 64, 128, 1, 1)
sep_conv_8_2 = seperate_convolution(sep_conv_8_1, 3, 128, 128, 1, 1)
sep_conv_4_1 = seperate_convolution(sep_conv_8_2, 3, 128, 256, 1, 1)
sep_conv_4_2 = seperate_convolution(sep_conv_4_1, 3, 256, 256, 1, 1)
sep_conv_2 = seperate_convolution(sep_conv_4_2, 3, 256, 512, 2, 1)
for i in range(5):
    sep_conv_2 = seperate_convolution(sep_conv_2, 3, 512, 512, 1, 1)
sep_conv_1_1 = seperate_convolution(sep_conv_2, 3, 512, 1024, 1, 1)
sep_conv_1_2 = seperate_convolution(sep_conv_1_1, 3, 1024, 1024, 1, 1)
avg_pool = keras.layers.AveragePooling2D((8, 8), strides=(1, 1), padding='valid')(sep_conv_1_2)
avg_pool = keras.layers.Flatten()(avg_pool)
fc = keras.layers.Dense(100, 'softmax')(avg_pool)



In [ ]:
# 모델 생성
model = tf.keras.Model(inputs=inputs, outputs=fc)

# 모델 컴파일
optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

In [15]:
# 모델 
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

 717/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.3389 - loss: 2.5400

In [ ]:
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")